# Rapid Analysis performance
Craig Lage - 16-Jan-26

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString

In [ ]:
dayObss = [20260112, 20260115]
imageTypes = ['acq', 'science', 'cwfs', 'engtest']
fig, axs = plt.subplots(1, 2, figsize=(15,5))

for dayObs in dayObss:
    perfTable = pd.read_json(f'/project/rubintv/raPerformance/sidecar_metadata/dayObs_{dayObs}.json').T
    perfTable = perfTable.sort_index()
    imageTable = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
    imageTable = imageTable.sort_index()
    mergedTable = pd.merge(imageTable, perfTable, left_index=True, right_index=True, how='inner')
    mergedTable = mergedTable[mergedTable['Image type_x'].isin(imageTypes)]
    axs[0].set_title(f"Shutter close to ISR start trend {dayObs}", fontsize=18)
    axs[0].scatter(mergedTable.index, mergedTable['ISR start time (shutter)'], label=dayObs)
    axs[0].set_xlabel("SeqNum")
    axs[0].set_ylabel("Seconds")
    #subset_of_dates = mergedTable['Date begin'].iloc[::10]
    axs[0].set_ylim(0,30)
    #axs[0].set_xticks(subset_of_dates)
    #axs[0].set_xticklabels(subset_of_dates, rotation=90)
    axs[1].hist(perfTable['ISR start time (shutter)'].values, label=dayObs, histtype='step', lw=2)
    axs[1].set_title(f"Shutter close to ISR start {dayObs}", fontsize=18)
    axs[1].set_xlabel("Seconds")

axs[0].legend()
axs[1].legend()
plt.savefig(f"/home/cslage/MTAOS/RA_performance/RA_Performance_16Jan26",
            bbox_inches='tight', pad_inches=1.2)

# This one uses "Readout end to isr start"

In [ ]:
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString

imageTypes = ['acq', 'science', 'cwfs', 'engtest']

startDay = 20251101
endDay = 20260112
dayObs = startDay
times = []
delays = []
dayObsVals = []
dayObsNames = []
while dayObs <= endDay:
    if dayObs == 20251207:
        dayObs = calcNextDay(dayObs)
        continue
    try:
        perfTable = pd.read_json(f'/project/rubintv/raPerformance/sidecar_metadata/dayObs_{dayObs}.json').T
        perfTable = perfTable.sort_index()
        imageTable = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        imageTable = imageTable.sort_index()
        mergedTable = pd.merge(imageTable, perfTable, left_index=True, right_index=True, how='inner')
        mergedTable = mergedTable[mergedTable['Image type_x'].isin(imageTypes)]
        thisDayDelays = []
        for i in range(len(mergedTable)):
            try:
                delay = mergedTable['Readout end to isr start'].iloc[i]
                if not np.isnan(delay):
                    times.append(Time(mergedTable['Date begin'].iloc[i], scale='tai').unix_tai)
                    delays.append(delay)
                    thisDayDelays.append(delay)
            except:
                continue
    except:
        print(f"{dayObs} failed!")
        dayObs = calcNextDay(dayObs)
        continue
    print(len(times), len(delays), np.nanmedian(thisDayDelays))
    print(f"Finished {dayObs}") 
    dayObsTime = f"{dayObsIntToString(dayObs)}T12:00:00"
    dayObsVals.append(Time(dayObsTime, format='isot').unix_tai)
    dayObsNames.append(dayObsIntToString(dayObs))
    dayObs = calcNextDay(dayObs)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.scatter(times, delays)
ax.set_ylim(0,40)
ax.set_xticks(dayObsVals[::2])
ax.set_xticklabels(dayObsNames[::2], rotation=90)
ax.set_ylabel('Readout end to isr start (seconds)', fontsize=12)
ax.set_title('Readout end to isr start,  imgType in (science, acq, cwfs, engtest)', fontsize=18)
plt.savefig(f"/home/cslage/MTAOS/RA_performance/RA_Performance_Trend_{startDay}_{endDay}.png",
            bbox_inches='tight', pad_inches=1.2)

# This one uses "ISR start time (shutter)"

In [ ]:
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString

imageTypes = ['acq', 'science', 'cwfs', 'engtest']

startDay = 20251101
endDay = 20260115
dayObs = startDay
times = []
delays = []
dayObsVals = []
dayObsNames = []
while dayObs <= endDay:
    if dayObs == 20251207:
        dayObs = calcNextDay(dayObs)
        continue
    try:
        perfTable = pd.read_json(f'/project/rubintv/raPerformance/sidecar_metadata/dayObs_{dayObs}.json').T
        perfTable = perfTable.sort_index()
        imageTable = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        imageTable = imageTable.sort_index()
        mergedTable = pd.merge(imageTable, perfTable, left_index=True, right_index=True, how='inner')
        mergedTable = mergedTable[mergedTable['Image type_x'].isin(imageTypes)]
        thisDayDelays = []
        for i in range(len(mergedTable)):
            try:
                delay = mergedTable['ISR start time (shutter)'].iloc[i]
                if not np.isnan(delay):
                    times.append(Time(mergedTable['Date begin'].iloc[i], scale='tai').unix_tai)
                    delays.append(delay)
                    thisDayDelays.append(delay)
            except:
                continue
    except:
        print(f"{dayObs} failed!")
        dayObs = calcNextDay(dayObs)
        continue
    print(len(times), len(delays), np.nanmedian(thisDayDelays))
    print(f"Finished {dayObs}") 
    dayObsTime = f"{dayObsIntToString(dayObs)}T12:00:00"
    dayObsVals.append(Time(dayObsTime, format='isot').unix_tai)
    dayObsNames.append(dayObsIntToString(dayObs))
    dayObs = calcNextDay(dayObs)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.scatter(times, delays)
ax.set_ylim(0,40)
ax.set_xticks(dayObsVals[::2])
ax.set_xticklabels(dayObsNames[::2], rotation=90)
ax.set_ylabel('Shutter close to isr start (seconds)', fontsize=12)
ax.set_title('Shutter close to isr start,  imgType in (science, acq, cwfs, engtest)', fontsize=18)
plt.savefig(f"/home/cslage/MTAOS/RA_performance/RA_Performance_Trend_2_{startDay}_{endDay}.png",
            bbox_inches='tight', pad_inches=1.2)

# This one uses "Zernike delivery time (shutter)"

In [ ]:
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString

imageTypes = ['acq', 'science', 'cwfs', 'engtest']

startDay = 20251101
endDay = 20260115
dayObs = startDay
times = []
delays = []
dayObsVals = []
dayObsNames = []
while dayObs <= endDay:
    if dayObs == 20251207:
        dayObs = calcNextDay(dayObs)
        continue
    try:
        perfTable = pd.read_json(f'/project/rubintv/raPerformance/sidecar_metadata/dayObs_{dayObs}.json').T
        perfTable = perfTable.sort_index()
        imageTable = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
        imageTable = imageTable.sort_index()
        mergedTable = pd.merge(imageTable, perfTable, left_index=True, right_index=True, how='inner')
        mergedTable = mergedTable[mergedTable['Image type_x'].isin(imageTypes)]
        thisDayDelays = []
        for i in range(len(mergedTable)):
            try:
                delay = mergedTable['Zernike delivery time (shutter)'].iloc[i]
                if not np.isnan(delay):
                    times.append(Time(mergedTable['Date begin'].iloc[i], scale='tai').unix_tai)
                    delays.append(delay)
                    thisDayDelays.append(delay)
            except:
                continue
    except:
        print(f"{dayObs} failed!")
        dayObs = calcNextDay(dayObs)
        continue
    print(len(times), len(delays), np.nanmedian(thisDayDelays))
    print(f"Finished {dayObs}") 
    dayObsTime = f"{dayObsIntToString(dayObs)}T12:00:00"
    dayObsVals.append(Time(dayObsTime, format='isot').unix_tai)
    dayObsNames.append(dayObsIntToString(dayObs))
    dayObs = calcNextDay(dayObs)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.scatter(times, delays)
ax.set_ylim(0,200)
ax.set_xticks(dayObsVals[::2])
ax.set_xticklabels(dayObsNames[::2], rotation=90)
ax.set_ylabel('Shutter close to Zernike delivery (seconds)', fontsize=12)
ax.set_title('Shutter close to Zernike delivery,  imgType in (science, acq, cwfs, engtest)', fontsize=18)
plt.savefig(f"/home/cslage/MTAOS/RA_performance/RA_Performance_Trend_3_{startDay}_{endDay}.png",
            bbox_inches='tight', pad_inches=1.2)